In [1]:
import os,glob
import numpy as np
import pandas as pd
import glob
import pyslha
import time
import progressbar as P
import matplotlib.pyplot as plt
import os,glob
import numpy as np
import seaborn as sns
from scipy.special import kn,zetac
import pyslha
import sys
sys.path.append('../')
from helper import getLLPs,getDisplacedJets
sns.set_style('ticks',{'font.family':'serif', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
sns.set_palette(sns.color_palette("Paired"))

delphesDir = os.path.abspath("../DelphesLLP")
os.environ['ROOT_INCLUDE_PATH'] = os.path.join(delphesDir,"external")

import ROOT

ROOT.gSystem.Load(os.path.join(delphesDir,"libDelphes.so"))

ROOT.gInterpreter.Declare('#include "classes/SortableObject.h"')
ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')


Welcome to JupyROOT 6.30/02


True

In [2]:
inputFile = list(glob.glob('../eventData/pp2BB1j_cms_scan_New/Events/run_02/cdfo_*.root'))[0]

In [6]:
f = ROOT.TFile(inputFile,'read')
tree = f.Get("Delphes")
nevts = tree.GetEntries()

for ievt in range(nevts):
    tree.GetEntry(118)
    jets = tree.Jet
    llps = getLLPs(tree.bsm,tree.bsmDirectDaughters,tree.bsmFinalDaughters,mothers=tree.bsmMothers)
    # llps = getLLPs(tree.bsm,tree.bsmDirectDaughters,[])
    jetsDisp = getDisplacedJets(jets,llps)
    missingET = tree.MissingET.At(0).MET
    genmiss = tree.GenMissingET.At(0).MET
    dmmiss = tree.DMMissingET.At(0).MET
    break
    if (jets.GetEntries() != jetsD.GetEntries()):
        print(ievt)
        break

In [7]:
print(llps)
for llp in llps:
    print(llp,llp.getCharge())

[1000005 (106), -1000005 (106)]
1000005 (106) -1.0
-1000005 (106) 0.0


In [45]:
for j in jets:
    print(j.PT,j.Constituents.GetEntries())

52.73524475097656 21


In [48]:
pTot = np.zeros(4)
for llp in llps:
    print(llp.PID,llp.Status,llp.PT,llp.Eta,llp.r_decay)
    pTot += np.array([llp.Px,llp.Py,llp.Pz,llp.E])
print(np.linalg.norm(pTot[:2]))

1000522 1 123.52571105957031 0.4662858247756958 inf
-1005113 1 152.25868225097656 1.4389843940734863 inf
60.23372458249569


In [13]:
for j in jetsDisp:
    print(j.PT,j.Constituents.GetEntries())

In [31]:
for ij in range(jets.GetEntries()):
    print('\nJEts:')
    j = jets.At(ij)
    jD = jetsD.At(ij)
    print(j.Particles.GetEntries(),jD.Particles.GetEntries())
    print(j.Constituents.GetEntries(),jD.Constituents.GetEntries())
    print(j.PT,jD.PT)
    print('Const:')
    pTot = np.zeros(4)
    for ic in range(j.Particles.GetEntries()):
        p = j.Particles.At(ic)
        pTot += np.array([p.E,p.Px,p.Py,p.Pz])
        rdisp = np.sqrt(p.X**2 + p.Y**2)
        if rdisp < 2.0:
            continue
        print(p.PID,p.PT,p.Charge,'%1.1f' %(rdisp))
    print('  pT = ',np.sqrt(np.dot(pTot[1:3],pTot[1:3])))
    print('ConstD:')
    pTot = np.zeros(4)
    for ic in range(jD.Particles.GetEntries()):
        p = jD.Particles.At(ic)
        pTot += np.array([p.E,p.Px,p.Py,p.Pz])
        rdisp = np.sqrt(p.X**2 + p.Y**2)
        if rdisp < 4.0:
            continue
        print(p.PID,p.PT,p.Charge,'%1.1f' %(rdisp))    
    print('  pT = ',np.sqrt(np.dot(pTot[1:3],pTot[1:3])))


JEts:
29 15
28 15
314.2334289550781 60.73228073120117
Const:
321 0.5215311646461487 1 29.2
-321 0.46100255846977234 -1 29.2
211 2.2757747173309326 1 29.2
  pT =  280.3249214858861
ConstD:
211 1.269769549369812 1 56.2
-211 1.152583360671997 -1 56.2
211 3.441673755645752 1 81.7
  pT =  14.345142035633863

JEts:
31 10
31 10
161.031982421875 40.71505355834961
Const:
-321 2.121795654296875 -1 81.7
211 3.441673755645752 1 81.7
211 2.0287418365478516 1 135.0
211 3.5668108463287354 1 81.7
  pT =  159.78955062355158
ConstD:
  pT =  3.778586475173732

JEts:
14 9
14 9
69.44495391845703 32.48310089111328
Const:
  pT =  103.66892563101925
ConstD:
211 3.5668108463287354 1 81.7
  pT =  34.9850728696782


PT,Eta,Phi = 33.32, -4.31, -0.18


In [17]:
for llp in llps:
    for d in llp.directDaughters:
        print(d.PID,'PT,Eta,Phi = %1.2f, %1.2f, %1.2f' %(d.PT,d.Eta,d.Phi))

-5 PT,Eta,Phi = 1.39, 0.52, -0.08
1000022 PT,Eta,Phi = 39.78, 2.38, 1.31
5 PT,Eta,Phi = 2.92, 1.69, -1.53
1000022 PT,Eta,Phi = 64.36, -0.05, -2.44
